# lora 생성

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from peft import get_peft_model, LoraConfig, TaskType

In [4]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, 
    r=8, 
    lora_alpha=16, 
    target_modules=["query", "key", "value", "dense"],
    lora_dropout=0.1
)

In [5]:
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
model = AutoModelForSequenceClassification.from_pretrained('klue/bert-base', num_labels=43)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
dataset = load_dataset('csv', data_files={'train': 'DoWADO/AI/tfs_train_1.csv'})
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets.set_format("torch")

In [7]:
model = get_peft_model(model, lora_config)

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [8]:
training_args = TrainingArguments(
    output_dir='DoWADO/AI/results_lora_model',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
)

In [9]:
trainer.train()

Step,Training Loss
500,3.167800
1000,1.833400
1500,1.385400
2000,1.200900
2500,1.070200
3000,1.019100


TrainOutput(global_step=3250, training_loss=1.5656431790865384, metrics={'train_runtime': 2139.9964, 'train_samples_per_second': 24.299, 'train_steps_per_second': 1.519, 'total_flos': 1.3906053537792e+16, 'train_loss': 1.5656431790865384, 'epoch': 10.0})

In [10]:
model.save_pretrained("DoWADO/AI/lora/lora_model")
tokenizer.save_pretrained("DoWADO/AI/lora/lora_tokenizer")

('DoWADO/AI/lora/lora_tokenizer/tokenizer_config.json',
 'DoWADO/AI/lora/lora_tokenizer/special_tokens_map.json',
 'DoWADO/AI/lora/lora_tokenizer/vocab.txt',
 'DoWADO/AI/lora/lora_tokenizer/added_tokens.json',
 'DoWADO/AI/lora/lora_tokenizer/tokenizer.json')

# lora와 원본 모델 합치기

In [10]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel, PeftConfig

# 원본 모델과 토크나이저 불러오기
model_name = "klue/bert-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=43)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# safetensor 파일 경로 설정
safetensor_path = "DoWADO/AI/lora/lora_model"

# LoRA 방식으로 학습된 가중치 불러오기
lora_config = PeftConfig.from_pretrained(safetensor_path)
lora_model = PeftModel.from_pretrained(model, safetensor_path, config=lora_config)

# 원본 모델에 LoRA 가중치 적용
model.load_state_dict(lora_model.state_dict(), strict=False)

# 적용된 모델 확인
for name, param in model.named_parameters():
    if "lora_" in name:
        print(name, param)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


bert.encoder.layer.0.attention.self.query.lora_A.default.weight Parameter containing:
tensor([[ 0.0004,  0.0182,  0.0041,  ..., -0.0074, -0.0115,  0.0261],
        [ 0.0232,  0.0069,  0.0290,  ..., -0.0151, -0.0414,  0.0078],
        [-0.0261, -0.0036, -0.0002,  ...,  0.0209,  0.0165,  0.0454],
        ...,
        [-0.0075, -0.0186, -0.0200,  ..., -0.0350,  0.0141,  0.0238],
        [ 0.0041,  0.0199, -0.0024,  ..., -0.0009,  0.0309,  0.0280],
        [ 0.0336,  0.0421, -0.0072,  ..., -0.0219, -0.0160, -0.0078]])
bert.encoder.layer.0.attention.self.query.lora_B.default.weight Parameter containing:
tensor([[-2.1295e-03,  1.1604e-04,  6.9179e-04,  ...,  2.3041e-03,
          3.7055e-03,  1.2358e-04],
        [ 2.6309e-03, -6.0687e-03, -8.5320e-03,  ...,  1.3428e-03,
         -8.6273e-03, -1.2065e-03],
        [ 4.1978e-03,  5.2147e-03,  3.8513e-04,  ..., -5.7858e-04,
          5.3502e-03, -1.1400e-03],
        ...,
        [ 1.1713e-02,  6.9995e-03,  4.1999e-03,  ..., -5.3764e-03,
     

In [12]:
model.save_pretrained('DoWADO/AI/lora/combined_model')
tokenizer.save_pretrained('DoWADO/AI/lora/combined_tokenizer')

('DoWADO/AI/lora/combined_tokenizer/tokenizer_config.json',
 'DoWADO/AI/lora/combined_tokenizer/special_tokens_map.json',
 'DoWADO/AI/lora/combined_tokenizer/vocab.txt',
 'DoWADO/AI/lora/combined_tokenizer/added_tokens.json',
 'DoWADO/AI/lora/combined_tokenizer/tokenizer.json')